<a href="https://colab.research.google.com/github/Minteb/Hyperspectral-Image-Classification-using-Random-Forest-Algorithm/blob/main/docs/workshops/Crop_Mapping_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install geemap

## Import libraries

In [49]:
import ee
import geemap

In [50]:
ee.Authenticate()
ee.Initialize(project="spiaproject")

In [51]:
Map = geemap.Map()

esa_wms = "https://services.terrascope.be/wms/v2"  # The WMS URL
tcc_layer = "WORLDCOVER_2020_S2_TCC"  # The true color composite imagery
fcc_layer = "WORLDCOVER_2020_S2_FCC"  # The false color composite imagery
map_layer = "WORLDCOVER_2020_MAP"  # The land cover classification map

Map.add_wms_layer(esa_wms, layers=tcc_layer, name="True Color", attribution="ESA")
Map.add_wms_layer(esa_wms, layers=fcc_layer, name="False Color", attribution="ESA")
Map.add_wms_layer(esa_wms, layers=map_layer, name="Classification", attribution="ESA")

Map.add_legend(title="ESA Land Cover", builtin_legend="ESA_WorldCover")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Using Earth Engine

- [EAS WroldCover in the Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100)

The European Space Agency (ESA) WorldCover 10 m 2020 product provides a global land cover map for 2020 at 10 m resolution based on Sentinel-1 and Sentinel-2 data. The WorldCover product comes with 11 land cover classes and has been generated in the framework of the ESA WorldCover project, part of the 5th Earth Observation Envelope Programme (EOEP-5) of the European Space Agency.

In [52]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

esa = ee.ImageCollection("ESA/WorldCover/v100").first()
esa_vis = {"bands": ["Map"]}

Map.addLayer(esa, esa_vis, "ESA Land Cover")
Map.add_legend(title="ESA Land Cover", builtin_legend="ESA_WorldCover")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Creating charts

In [53]:
histogram = geemap.image_histogram(
    esa, scale=1000, x_label="Land Cover Type", y_label="Area (km2)"
)
histogram

In [54]:
df = geemap.image_histogram(esa, scale=1000, return_df=True)
df

,key,value
0,10,6.139810e+07
1,20,1.211905e+07
2,30,4.470917e+07
3,40,1.665006e+07
4,50,8.475490e+05
5,60,2.858194e+07
6,70,8.461224e+06
7,80,3.219519e+07
8,90,4.268191e+06
9,95,1.941690e+05


In [55]:
esa_labels = list(geemap.builtin_legends["ESA_WorldCover"].keys())
esa_labels

['10 Trees',
 '20 Shrubland',
 '30 Grassland',
 '40 Cropland',
 '50 Built-up',
 '60 Barren / sparse vegetation',
 '70 Snow and ice',
 '80 Open water',
 '90 Herbaceous wetland',
 '95 Mangroves',
 '100 Moss and lichen']

In [56]:
df["label"] = esa_labels
df

,key,value,label
0,10,6.139810e+07,10 Trees
1,20,1.211905e+07,20 Shrubland
2,30,4.470917e+07,30 Grassland
3,40,1.665006e+07,40 Cropland
4,50,8.475490e+05,50 Built-up
5,60,2.858194e+07,60 Barren / sparse vegetation
6,70,8.461224e+06,70 Snow and ice
7,80,3.219519e+07,80 Open water
8,90,4.268191e+06,90 Herbaceous wetland
9,95,1.941690e+05,95 Mangroves


In [57]:
round(df["value"].sum() / 1e6, 2)

np.float64(217.06)

In [58]:
geemap.bar_chart(
    df, x="label", y="value", x_label="Land Cover Type", y_label="Area (km2)"
)

In [59]:
geemap.pie_chart(df, names="label", values="value", height=500)

### Adding Administrative Boundaries

In [60]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))
africa = countries.filter(ee.Filter.eq("CONTINENT", "Africa"))
style = {"fillColor": "00000000"}
Map.addLayer(countries.style(**style), {}, "Countries", False)
Map.addLayer(africa.style(**style), {}, "Africa")
Map.centerObject(africa)
Map

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [61]:
# Load countries and filter Ethiopia
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))
ethiopia = countries.filter(ee.Filter.eq("ADMIN", "Ethiopia"))

In [62]:
# Load ESA WorldCover 2021 LULC dataset
lulc = ee.ImageCollection("ESA/WorldCover/v100").first().select('Map')


In [63]:
# Clip the LULC image to Ethiopia boundary
ethiopia_lulc = lulc.clip(ethiopia)


In [64]:
# Set export parameters
task = ee.batch.Export.image.toDrive(
    image=ethiopia_lulc,
    description='Ethiopia_LULC_2021',
    folder='EarthEngine',
    fileNamePrefix='ethiopia_lulc_2021',
    region=ethiopia.geometry(),
    scale=10,
    maxPixels=1e13
)

task.start()


### Extracting Croplands

In [18]:
cropland = esa.eq(40).clipToCollection(africa).selfMask()
Map.addLayer(cropland, {"palette": ["f096ff"]}, "Cropland")
Map.show_layer(name="ESA Land Cover", show=False)

### Zonal Statistics

In [19]:
geemap.zonal_stats(cropland, africa, "esa_cropland.csv", stat_type="SUM", scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esa_cropland.csv


In [20]:
df = geemap.csv_to_df("esa_cropland.csv")
df.head()

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,5235.411765,66010.0,CD,16,COD,Africa,83301151,5. Low income,Middle Africa,0000000000000000000b,Dem. Rep. Congo
1,40286.113725,30590.0,TD,14,TCD,Africa,12075985,5. Low income,Middle Africa,0000000000000000000f,Chad
2,279.831373,3206.0,CF,13,CAF,Africa,5625118,5. Low income,Middle Africa,00000000000000000042,Central African Rep.
3,175513.933333,150600.0,TZ,16,TZA,Africa,53950935,5. Low income,Eastern Africa,00000000000000000001,Tanzania
4,8584.458824,4719.0,SO,13,SOM,Africa,7531386,5. Low income,Eastern Africa,0000000000000000000c,Somalia


In [21]:
geemap.bar_chart(
    df, x="NAME", y="sum", max_rows=30, x_label="Country", y_label="Area (km2)"
)

In [22]:
geemap.pie_chart(df, names="NAME", values="sum", max_rows=20, height=500)

## ESRI GLobal Land Cover

The ESRI GLobal Land Cover dataset is a global map of land use/land cover (LULC) derived from ESA Sentinel-2 imagery at 10m resolution. Each year is generated from Impact Observatory’s deep learning AI land classification model used a massive training dataset of billions of human-labeled image pixels developed by the National Geographic Society. The global maps were produced by applying this model to the Sentinel-2 scene collection on Microsoft’s Planetary Computer, processing over 400,000 Earth observations per year.

- https://livingatlas.arcgis.com/landcover/
- https://www.arcgis.com/home/item.html?id=d3da5dd386d140cf93fc9ecbf8da5e31
- https://samapriya.github.io/awesome-gee-community-datasets/projects/S2TSLULC/

### Using Awesome GEE Community Datasets

In [23]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

esri = ee.ImageCollection(
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
)

esri_2017 = esri.filterDate("2017-01-01", "2017-12-31").mosaic()
esri_2019 = esri.filterDate("2019-01-01", "2019-12-31").mosaic()
esri_2021 = esri.filterDate("2021-01-01", "2021-12-31").mosaic()
esri_2023 = esri.filterDate("2023-01-01", "2023-12-31").mosaic()
esri_2024 = esri.filterDate("2024-01-01", "2024-12-31").mosaic()

esri_vis = {"min": 1, "max": 11, "palette": "esri_lulc"}

Map.addLayer(esri_2017, esri_vis, "ESRI LULC 2017")
Map.addLayer(esri_2018, esri_vis, "ESRI LULC 2018")
Map.addLayer(esri_2019, esri_vis, "ESRI LULC 2019")
Map.addLayer(esri_2020, esri_vis, "ESRI LULC 2020")
Map.addLayer(esri_2021, esri_vis, "ESRI LULC 2021")

Map.add_legend(title="ESRI Land Cover", builtin_legend="ESRI_LandCover_TS")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Using Timeseries Inspector

In [24]:
images = ee.List([esri_2017, esri_2018, esri_2019, esri_2020, esri_2021])
collection = ee.ImageCollection.fromImages(images)

In [25]:
years = [str(year) for year in range(2017, 2022)]
years

['2017', '2018', '2019', '2020', '2021']

In [26]:
Map = geemap.Map()
Map.ts_inspector(collection, years, esri_vis, width="80px")
Map.add_legend(title="ESRI Land Cover", builtin_legend="ESRI_LandCover_TS")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=Layo…

### Extracting Croplands

In [27]:
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))
africa = countries.filter(ee.Filter.eq("CONTINENT", "Africa"))

In [28]:
cropland_col = collection.map(lambda img: img.eq(5).clipToCollection(africa).selfMask())
cropland_ts = cropland_col.toBands().rename(years)

In [29]:
Map = geemap.Map()

style = {"fillColor": "00000000"}
Map.addLayer(cropland_col.first(), {"palette": ["#ab6c28"]}, "first")
Map.addLayer(countries.style(**style), {}, "Countries", False)
Map.addLayer(africa.style(**style), {}, "Africa")
Map.centerObject(africa)

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [30]:
cropland_ts.bandNames().getInfo()

['2017', '2018', '2019', '2020', '2021']

### Zonal Statistics

In [31]:
geemap.zonal_stats(
    cropland_ts, africa, "esri_cropland.csv", stat_type="SUM", scale=1000
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esri_cropland.csv


In [32]:
df = geemap.csv_to_df("esri_cropland.csv")
df.head()

,2017,2018,2019,2020,2021,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,6876.454902,8326.317647,6269.674510,6023.917647,7061.415686,66010.0,CD,16,COD,Africa,83301151,5. Low income,Middle Africa,0000000000000000000b,Dem. Rep. Congo
1,19846.988235,17226.945098,16099.921569,18987.521569,19192.054902,30590.0,TD,14,TCD,Africa,12075985,5. Low income,Middle Africa,0000000000000000000f,Chad
2,360.447059,501.458824,576.321569,717.247059,667.133333,3206.0,CF,13,CAF,Africa,5625118,5. Low income,Middle Africa,00000000000000000042,Central African Rep.
3,46271.152941,45253.674510,50515.698039,55429.341176,58143.333333,150600.0,TZ,16,TZA,Africa,53950935,5. Low income,Eastern Africa,00000000000000000001,Tanzania
4,6925.113725,8671.768627,8718.145098,11651.392157,10313.219608,4719.0,SO,13,SOM,Africa,7531386,5. Low income,Eastern Africa,0000000000000000000c,Somalia


In [33]:
geemap.bar_chart(df, x="NAME", y=years, max_rows=20, legend_title="Year")

In [34]:
geemap.pie_chart(df, names="NAME", values="2020", max_rows=20, height=500)

### Analyzing Cropland Gain and Loss

In [35]:
Map = geemap.Map()
Map.add_basemap("HYBRID")

cropland_2017 = esri_2017.eq(5).selfMask()
cropland_2021 = esri_2021.eq(5).selfMask()

cropland_gain = esri_2017.neq(5).And(esri_2021.eq(5)).selfMask()
cropland_loss = esri_2017.eq(5).And(esri_2021.neq(5)).selfMask()

Map.addLayer(cropland_2017, {"palette": "brown"}, "Cropland 2017", False)
Map.addLayer(cropland_2021, {"palette": "cyan"}, "Cropland 2021", False)

Map.addLayer(cropland_gain, {"palette": "yellow"}, "Cropland gain")
Map.addLayer(cropland_loss, {"palette": "red"}, "Cropland loss")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [36]:
geemap.zonal_stats(
    cropland_gain,
    countries,
    "esri_cropland_gain.csv",
    stat_type="SUM",
    scale=1000,
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esri_cropland_gain.csv


In [37]:
df = geemap.csv_to_df("esri_cropland_gain.csv")
df.head()

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,1004.886275,25810.0,TJ,13,TJK,Asia,8468555,5. Low income,Central Asia,00000000000000000068,Tajikistan
1,1804.423529,21010.0,KG,13,KGZ,Asia,5789122,5. Low income,Central Asia,00000000000000000069,Kyrgyzstan
2,4681.835294,40000.0,KP,15,PRK,Asia,25248140,5. Low income,Eastern Asia,0000000000000000005f,North Korea
3,10450.321569,628400.0,BD,17,BGD,Asia,157826578,5. Low income,Southern Asia,00000000000000000063,Bangladesh
4,896.827451,71520.0,NP,15,NPL,Asia,29384297,5. Low income,Southern Asia,00000000000000000065,Nepal


In [38]:
geemap.bar_chart(
    df,
    x="NAME",
    y="sum",
    max_rows=30,
    x_label="Country",
    y_label="Area (km2)",
    title="Cropland Gain",
)

In [39]:
geemap.pie_chart(
    df, names="NAME", values="sum", max_rows=30, height=500, title="Cropland Gain"
)

In [40]:
geemap.zonal_stats(
    cropland_loss,
    countries,
    "esri_cropland_loss.csv",
    stat_type="SUM",
    scale=1000,
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/esri_cropland_loss.csv


In [41]:
df = geemap.csv_to_df("esri_cropland_loss.csv")
df.head()

,sum,GDP_MD_EST,ISO_A2,POP_RANK,ISO_A3,CONTINENT,POP_EST,INCOME_GRP,SUBREGION,system:index,NAME
0,952.619608,25810.0,TJ,13,TJK,Asia,8468555,5. Low income,Central Asia,00000000000000000068,Tajikistan
1,2303.243137,21010.0,KG,13,KGZ,Asia,5789122,5. Low income,Central Asia,00000000000000000069,Kyrgyzstan
2,4799.721569,40000.0,KP,15,PRK,Asia,25248140,5. Low income,Eastern Asia,0000000000000000005f,North Korea
3,8731.494118,628400.0,BD,17,BGD,Asia,157826578,5. Low income,Southern Asia,00000000000000000063,Bangladesh
4,1767.266667,71520.0,NP,15,NPL,Asia,29384297,5. Low income,Southern Asia,00000000000000000065,Nepal


In [42]:
geemap.bar_chart(
    df,
    x="NAME",
    y="sum",
    max_rows=30,
    x_label="Country",
    y_label="Area (km2)",
    title="Cropland Loss",
)

In [43]:
geemap.pie_chart(
    df, names="NAME", values="sum", max_rows=30, height=500, title="Cropland Loss"
)

## Dynamic World Land Cover

Dynamic World is a near realtime 10m resolution global land use land cover dataset, produced using deep learning, freely available and openly licensed. As a result of leveraging a novel deep learning approach, based on Sentinel-2 Top of Atmosphere, Dynamic World offers global land cover updating every 2-5 days depending on location.

- [Dynamic World Website](https://www.dynamicworld.app/)
- [Dynamic World datasets on Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1)

### Classification and Probability

In [48]:
Map = geemap.Map()

region = ee.Geometry.BBox(-179, -89, 179, 89)
start_date = "2021-01-01"
end_date = "2022-01-01"

dw_class = geemap.dynamic_world(region, start_date, end_date, return_type="class")
dw = geemap.dynamic_world(region, start_date, end_date, return_type="hillshade")

dw_vis = {"min": 0, "max": 8, "palette": "dw"}

Map.addLayer(dw_class, dw_vis, "DW Land Cover", False)
Map.addLayer(dw, {}, "DW Land Cover Hillshade")

Map.add_legend(title="Dynamic World Land Cover", builtin_legend="Dynamic_World")
Map.setCenter(-88.9088, 43.0006, 12)
Map

Map(center=[43.0006, -88.9088], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

### ESA Land Cover vs. Dynamic World

In [47]:
Map = geemap.Map(center=[39.3322, -106.7349], zoom=10)

left_layer = geemap.ee_tile_layer(esa, esa_vis, "ESA Land Cover")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World Land Cover")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="ESA Land Cover", builtin_legend="ESA_WorldCover", position="bottomleft"
)
Map.add_legend(
    title="Dynamic World Land Cover",
    builtin_legend="Dynamic_World",
    position="bottomright",
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

Map(center=[43.0006, -88.9088], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### ESRI Land Cover vs. Dynamic World

In [46]:
Map = geemap.Map(center=[-89.3998, 43.0886], zoom=10)

left_layer = geemap.ee_tile_layer(esri_2021, esri_vis, "ESRI Land Cover")
right_layer = geemap.ee_tile_layer(dw, {}, "Dynamic World Land Cover")

Map.split_map(left_layer, right_layer)
Map.add_legend(
    title="ESRI Land Cover", builtin_legend="ESRI_LandCover", position="bottomleft"
)
Map.add_legend(
    title="Dynamic World Land Cover",
    builtin_legend="Dynamic_World",
    position="bottomright",
)
Map.setCenter(-88.9088, 43.0006, 12)

Map

Map(center=[43.0006, -88.9088], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…